<a href="https://colab.research.google.com/github/Zr695/IMLO-OpenAssessment/blob/main/IMLO_OpenAssessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.autograd import Variable

#I was playing around with normalisation using mean and standard deviation
# def mean_std(loader):
#     num_pixels = 0
#     mean = 0.0
#     std = 0.0
#     for images, _ in loader:
#         batch_size, num_channels, height, width = images.shape
#         num_pixels += batch_size * height * width
#         mean += images.mean(axis=(0, 2, 3)).sum()
#         std += images.std(axis=(0, 2, 3)).sum()
#     mean /= num_pixels
#     std /= num_pixels
#     return mean, std

# mean, std = mean_std(initial_dataloader)

data_transform = transforms.Compose([
    transforms.Resize((500, 500)),
    transforms.ToTensor(),
    ])

training_data = datasets.Flowers102(
    root="data",
    split='train',
    download=True,
    transform=data_transform
)
validation_data = datasets.Flowers102(
    root="data",
    split='val',
    download=True,
    transform=data_transform
)
test_data = datasets.Flowers102(
    root="data",
    split='test',
    download=True,
    transform=data_transform
)


train_dataloader = DataLoader(training_data, batch_size=64)
validate_dataloader = DataLoader(validation_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(500*500*3, 512),
            nn.ReLU(),
            # nn.BatchNorm2D(), #Working on starting to change the layers
            # nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 102),
        )

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]

    def forward(self, x):
        x = self.flatten(x)
        logits = self.layers(x)
        return logits

model = NeuralNetwork()

In [3]:
learning_rate = 0.001
batch_size = 64
epochs = 10

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    # train_loop(train_dataloader, model, loss_fn, optimizer)
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")